In [14]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
np.random.seed(0)
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
test = test.fillna(0)

In [3]:
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]
x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_test = np.asarray(test.iloc[:,-8:])

In [4]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = scaler.fit_transform(x_test)
x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

In [5]:
model = Sequential()
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
history = model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, 
          shuffle=True, epochs=5)

Train on 227768 samples, validate on 75923 samples
Epoch 1/5
227768/227768 [==============================] - 207s 909us/step - loss: 0.9977 - acc: 0.6363 - val_loss: 1.3370 - val_acc: 0.5544
Epoch 2/5
227768/227768 [==============================] - 207s 910us/step - loss: 0.7809 - acc: 0.7171 - val_loss: 1.4190 - val_acc: 0.5186
Epoch 3/5
227768/227768 [==============================] - 206s 905us/step - loss: 0.6910 - acc: 0.7517 - val_loss: 1.5565 - val_acc: 0.5209
Epoch 4/5
227768/227768 [==============================] - 203s 891us/step - loss: 0.6416 - acc: 0.7710 - val_loss: 1.5439 - val_acc: 0.5169
Epoch 5/5
227768/227768 [==============================] - 208s 913us/step - loss: 0.6035 - acc: 0.7842 - val_loss: 1.5968 - val_acc: 0.5277


In [ ]:
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
y_test_decode = np.argmax(y_test, axis=1)
print(accuracy_score(y_test_decode, predicted))

In [1]:
# evaluate the model
_, acc_train = model.evaluate(x_train, y_train, verbose=0)
_, acc_test = model.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (acc_train, acc_test))
# learning curves of model accuracy
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show()

NameError: name 'model' is not defined